In [1]:
!pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable


In [2]:
# 버전 출력 (cell 2)

from importlib.metadata import version

print("파이토치 버전:", version("torch"))
print("tiktoken 버전:", version("tiktoken"))

파이토치 버전: 2.9.1
tiktoken 버전: 0.12.0


In [3]:
# 파일 다운로드, 로드 및 샘플 출력 (cell 3~4)

import os
import requests

if not os.path.exists("the-verdict.txt"):
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
    file_path = "the-verdict.txt"

    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(response.content)

In [4]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("총 문자 개수:", len(raw_text))
print(raw_text[:99])

총 문자 개수: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [5]:
# 정규식 split 실험 (cell 5~11)

import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [6]:
# 정규표현식 패턴 전체를 감싸는 소괄호 ( )인 '캡처 그룹'을 쓰지 않는 경우 결과에 구분자가 사라짐
# 반면 패턴을 소괄호 ()로 감싸면 split 결과 안에 구분자도 포함
# re.split(r'[,.]', "Hello,world.") 
# 결과: ['Hello', 'world', '']
# cf) 마침표(.)를 만났을 때, 그 시점을 기준으로 나누면 오른쪽은 "아무것도 없음", 이는 빈 문자열('')로 간주
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [7]:
# 각 항목에서 공백 삭제 및 빈 문자열 제외
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [8]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text) # | or 연산
result = [item.strip() for item in result if item and item.strip()] # 단축 평가
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [9]:
result = [i for i in re.split(r'([,.:;?_!"()\']|--|\s)', text) if i and i.strip()] 
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [10]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item and item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [11]:
# vocab 출력, 확인용 print (cell 11~14)

print(len(preprocessed))

4690


In [12]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [13]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [14]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [15]:
# SimpleTokenizerV1 (cell 15~18)

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab  # ex) {"hello": 0, "world": 1, ",": 2, "!": 3}
        self.int_to_str = {i:s for s,i in vocab.items()}  # {i:s ...} : 뒤집어서 (숫자:문자열) 형태로 저장

    # 입력 텍스트를 정규식으로 잘게 쪼갠 뒤(Tokenize), 공백 제거 후 사전(Vocab)을 통해 숫자 리스트(IDs)로 변환
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)         
        preprocessed = [item.strip() for item in preprocessed if item and item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]  # token list를 vocab을 이용해 숫자로 변환
        return ids

    # 토큰 ID를 텍스트로 변환
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])  # 모든 단어 사이 공백 한 칸을 넣어 합칩
        # 정규식 코드 re.sub(r'패턴', r'바꿀내용', 원본)
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)  # s+ == 공백 여러 개
        return text

In [16]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [17]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [18]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [19]:
# special token vocab 구성 (cell 19~21)

# 알지 못하는 단어를 표현하는 "<|unk|>"
# 훈련에서 텍스트의 끝을 나타내기 위해 사용된 "<|endoftext|>"
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [20]:
len(vocab.items())

1132

In [21]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [22]:
# SimpleTokenizerV2 to use <unk>, <endoftext> token (cell 22~25)


class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [23]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

# <|endoftext|> 토큰은 데이터셋을 구성할 때 전처리 단계에서 수동으로(임의로) 삽입
text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [24]:
tokenizer.encode(text)  # ('<|endoftext|>', 1130)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [25]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

In [26]:
# tiktoken 및 allowed_special (cell 26~30)
# tiktoken is a high-performance BPE (Byte Pair Encoding) tokenizer
# <unk>는 모르는 단어들 모두 동일 토큰으로 처리, 고로 decode 함수 통해 기존 단어로 복원 불가
# 그렇기에 modern LLM은 <unk> 없는 방식으로 tokenizer 설계함
# byte pair encoding, BPE = 어휘사전에 없는 단어를 더 작은 부분단어나 개별 문자로 분할하여 처리


import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

In [27]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

# tiktoken은 기본적으로 special token이 텍스트에 직접 적혀 있으면 오류 발생
# 특수 토큰(special token)을 “토큰 그대로 허용해서 인식해라”는 설정 위해 allowed_special 사용
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [28]:
tokenizer.special_tokens_set

{'<|endoftext|>'}

In [29]:
print(tokenizer.encode(text, allowed_special='all'))  # 모든 special token을 텍스트 안에서 허용

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [30]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [31]:
# Sampling data with a sliding window (cell 31~35)
# 시퀀스 상의 다음 한 단어를 타깃(Target)으로 설정하여 모델이 순차적으로 단어를 생성


with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [32]:
enc_sample = enc_text[50:]

In [33]:
# 모델의 컨텍스트 길이(Context Length) 제한으로 인해 전체 텍스트를 일정 토큰 단위로 나누어 학습한다.
# 이때 잘라낸 한 줄기 토큰 덩어리를 청크(Chunk)라 한다.
# text chunk = 모델이 한 번에 볼 수 있는 입력 토큰들의 덩어리
# stride 때문에 chunk가 서로 겹치는 것은 일반적이고, GPT 학습에서는 오히려 필수적이다.
# 이를 통해 모델이 더 많은 “위치 기반 문맥 패턴”을 배울 수 있기 때문이다.
# 보통 GPT 학습(사전 학습(Pre-training) 단계)에서는 stride = 1 또는 GPU 메모리 때문에 최소 stride = max_length/2 정도를 사용한다.

context_size = 4  # window size, chunk size = 4 tokens

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]  # stride = 1  

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


In [34]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [35]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [36]:
# GPTDatasetV1 (cell 36) 


import torch
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    # max_length : 모델이 한 번에 볼 수 있는 context size
    # stride: sliding window우가 한 번에 옆으로 건너뛰는(이동하는) 토큰의 개수
    # A token divided by the sliding window in the input_ids and target_ids list continues to be appended.
    def __init__(self, txt, tokenizer, max_length, stride): 
        self.input_ids = []  # x (batch의 입력)
        self.target_ids = []  # y (next-token target)

        # Tokenize the entire text
        # context window 구성을 위한 최소 토큰 길이를 검증하고, 미달 시 assert로 실행을 강제 중단
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "토큰화된 입력의 개수는 적어도 max_length+1과 같아야 합니다."
        
        # sliding window로 max_length size의 데이터를 stride 간격만큼 겹쳐가며 시퀀스를 분할
        # 이후 텐서로 변환해 리스트에 저장
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1] 
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    # Dataset size
    # input_ids 개수 == chunk 개수, 즉, 몇 개의 (x, y)쌍이 있는지 반환
    def __len__(self): 
        return len(self.input_ids) 

    # # 특정 index의 sample을 반환
    def __getitem__(self, idx): 
        return self.input_ids[idx], self.target_ids[idx]

C:\Users\hjh04\AppData\Roaming\Python\Python313\site-packages\torch\_subclasses\functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [37]:
# create_dataloader_v1 (cell 37) 
# dataloader :모델 학습에 쓸 데이터를 잘라서, 정해진 크기(batch)로, 반복적으로 모델에게 공급해주는 도구 
# dataloder 생성 시, shuffle은 순서나 패턴을 통째로 외우는 것(Overfitting)을 방지하기 위해 사용
# 학습(Train) 단계에서는 True로 설정해 매번 다른 순서로 학습, 평가(Test/Validation) 단계에서는 결과 비교를 위해 False
# 고로 학습시에는 청크들의 등장 순서에 shuffle


# tokenizer 생성, dataset 생성, dataloader 생성
def create_dataloader_v1(txt, batch_size=4, max_length=256,  stride=128, shuffle=True, drop_last=True, num_workers=0):
    
    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")
    
    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    
    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,  # 모델 학습시에는 청크들의 등장 순서에 shuffle=True
        drop_last=drop_last,
        num_workers=num_workers  # 데이터 로딩 속도를 높이기 위해 사용하는 서브 프로세스(일꾼)의 수
    )

    return dataloader

In [38]:
# Test batch sample with create_dataloader_v1 (cell 38~41)


with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [39]:
# ex) batch_size=1 → (input_tensor 1개, target_tensor 1개)
# ex) batch_size=4 → (input_tensor 4개, target_tensor 4개) 즉, (x, y) 튜플이 넘어옴
# ex) batch_size=2 → (chunk1, chunk2) 묶어서 학습, (chunk3, chunk4) 묶어서 학습
# batch_size는 한 번의 학습 단계에서 모델이 동시에 처리할 청크(데이터 묶음)의 개수를 의미

# Instruction Fine-Tuning은 이미 언어를 다 배운 모델(pretrained language model)에게 대화하는 법을 가르치는 단계이다.
# 이는 pretrained language model을 사용자의 명령(instruction)을 수행하는 assistant-style 모델로 바꾸기 위해 
# Instruction–Response 쌍으로 supervised fine-tuning 하는 과정이다.
# 대부분의 Fine-Tuning 완성된 독립적인 instruction-respond 쌍이다.
# 고로 겹치는 chunk를 만드는 것은 의미 없는 중복 연산이며, 오히려 데이터 구조를 파괴하는 잘못된 방식이다.
# 그렇기에 Instruction Fine-Tuning은 stride = max_length 가 표준이다.
# 배치 간 중첩은 불필요한 연산이 증가하고, 특정 데이터에 과하게 치우치는 과적합(Overfitting)이 유발 될 수 있다.

dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=1, shuffle=False)

data_iter = iter(dataloader)
first_batch = next(data_iter)

print(first_batch)

[tensor([[  40,  367, 2885, 1464],
        [ 367, 2885, 1464, 1807]]), tensor([[ 367, 2885, 1464, 1807],
        [2885, 1464, 1807, 3619]])]


In [40]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[2885, 1464, 1807, 3619],
        [1464, 1807, 3619,  402]]), tensor([[1464, 1807, 3619,  402],
        [1807, 3619,  402,  271]])]


In [41]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("입력:\n", inputs)
print("\n타깃:\n", targets)

입력:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

타깃:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


In [42]:
# embedding layer sample (cell 42~45)
# torch.nn.Embedding을 통해 만들어지는 embedding layer는 LLM안에서 token ID → Dense(실수) 벡터로 변환하는 층아다.
# embedding layer는 숫자(ID)를 의미를 가진 실수 좌표(벡터)로 바꿔주는 거대한 조회 테이블(Look-up Table)이다.


input_ids = torch.tensor([2, 3, 5, 1])  # Tokenization 이후 token ID = 2, 3, 5, 1
vocab_size = 6  
output_dim = 3  # embedding size = 3

torch.manual_seed(123)  # seed를 고정 → 항상 같은 랜덤값, 재현성(reproducibility)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)  

In [43]:
# embedding_layer.weight는 전체 Embedding 행렬을 출력
print(embedding_layer.weight)  # 6x3 가중치 행렬

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [44]:
print(embedding_layer(torch.tensor([3])))  # Embedding 행렬의 네 번째 행

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [45]:
print(embedding_layer(input_ids))  # Look-up 연산 진행한 임베딩 층 결과

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


In [46]:
# token_embedding by inserting inputs from the data loader (cell 46~49)
# sampling data from the data loader, each sample's token is embedded into a 256D vector.


vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [47]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [48]:
print("토큰 ID:\n", inputs)
print("\n입력 크기:\n", inputs.shape)

토큰 ID:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

입력 크기:
 torch.Size([8, 4])


In [49]:
token_embeddings = token_embedding_layer(inputs)

print(token_embeddings.shape)  # batch_size=8, max_length = 4, output_dim = 256
print(token_embeddings)

torch.Size([8, 4, 256])
tensor([[[ 0.4913,  1.1239,  1.4588,  ..., -0.3995, -1.8735, -0.1445],
         [ 0.4481,  0.2536, -0.2655,  ...,  0.4997, -1.1991, -1.1844],
         [-0.2507, -0.0546,  0.6687,  ...,  0.9618,  2.3737, -0.0528],
         [ 0.9457,  0.8657,  1.6191,  ..., -0.4544, -0.7460,  0.3483]],

        [[ 1.5460,  1.7368, -0.7848,  ..., -0.1004,  0.8584, -0.3421],
         [-1.8622, -0.1914, -0.3812,  ...,  1.1220, -0.3496,  0.6091],
         [ 1.9847, -0.6483, -0.1415,  ..., -0.3841, -0.9355,  1.4478],
         [ 0.9647,  1.2974, -1.6207,  ...,  1.1463,  1.5797,  0.3969]],

        [[-0.7713,  0.6572,  0.1663,  ..., -0.8044,  0.0542,  0.7426],
         [ 0.8046,  0.5047,  1.2922,  ...,  1.4648,  0.4097,  0.3205],
         [ 0.0795, -1.7636,  0.5750,  ...,  2.1823,  1.8231, -0.3635],
         [ 0.4267, -0.0647,  0.5686,  ..., -0.5209,  1.3065,  0.8473]],

        ...,

        [[-1.6156,  0.9610, -2.6437,  ..., -0.9645,  1.0888,  1.6383],
         [-0.3985, -0.9235, -1.31

In [50]:
# pos_embedding_layer(position embedding, GPT-2) (cell 50~52)
# Token embeddings and positional embeddings are added to create input embeddings to be used for LLM.
# token_emb = nn.Embedding(vocab_size, d_model), pos_emb = nn.Embedding(context_length, d_model)
# nn.Embedding은 테이블(lookup table) 이다. 한 번 만들어지면 파라미터로 고정된고 forward마다 값이 바뀌지 않는다.
# 초기만 랜덤, 학습 중에 업데이트, 이후엔 고정이다. 
# token_vecs = token_emb(inputs)           # (batch, seq_len, d_model)
# positions = torch.arange(seq_len)
# pos_vecs  = pos_emb(positions)           # (seq_len, d_model)
# x = token_vecs + pos_vecs                # broadcast
# ex) positions = torch.arange(seq_len) -> tensor([0, 1, 2, 3, 4])


context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

print(pos_embedding_layer.weight)

Parameter containing:
tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       requires_grad=True)


In [51]:
# torch.arange(max_length) #  [0, 1, 2, ..., max_length-1] 
# Embedding(indices)는 테이블에서 indices에 해당하는 row들을 그대로 뽑아온다.
# pos_embedding_layer(torch.arange(context_length)) == pos_embedding_layer.weight

pos_embeddings = pos_embedding_layer(torch.arange(max_length))

print(pos_embeddings.shape)
print(pos_embeddings)

torch.Size([4, 256])
tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       grad_fn=<EmbeddingBackward0>)


In [52]:
input_embeddings = token_embeddings + pos_embeddings

print(input_embeddings.shape)
print(input_embeddings)

torch.Size([8, 4, 256])
tensor([[[ 2.2288,  0.5619,  0.8286,  ..., -0.6272, -0.2987,  0.8900],
         [ 2.0903, -0.4664, -0.0593,  ...,  0.9115, -1.0493, -1.6473],
         [-0.7158, -0.8304,  1.2494,  ...,  2.3952,  1.8773,  0.8051],
         [ 0.2703,  0.4029,  3.0514,  ...,  0.3595, -1.4548,  0.8310]],

        [[ 3.2835,  1.1749, -1.4150,  ..., -0.3281,  2.4332,  0.6924],
         [-0.2199, -0.9114, -0.1750,  ...,  1.5337, -0.1998,  0.1462],
         [ 1.5197, -1.4240,  0.4391,  ...,  1.0494, -1.4318,  2.3057],
         [ 0.2893,  0.8346, -0.1884,  ...,  1.9602,  0.8709,  0.8796]],

        [[ 0.9662,  0.0952, -0.4640,  ..., -1.0320,  1.6290,  1.7771],
         [ 2.4468, -0.2154,  1.4984,  ...,  1.8766,  0.5595, -0.1423],
         [-0.3856, -2.5393,  1.1556,  ...,  3.6157,  1.3267,  0.4944],
         [-0.2487, -0.5275,  2.0009,  ...,  0.2930,  0.5977,  1.3300]],

        ...,

        [[ 0.1219,  0.3991, -3.2740,  ..., -1.1921,  2.6637,  2.6728],
         [ 1.2438, -1.6436, -1.11